In [1]:
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString

In [2]:
filename = "North-East_American_Canada_goose_migration.csv"
df = pd.read_csv(filename)

In [3]:
geometry = []
for _, row in df.iterrows():
    geometry.append(Point(row['location-long'], row['location-lat']))

In [4]:
df = df.drop(['location-long', 'location-lat', 'event-id', 'visible', 'sensor-type', 'tag-local-identifier', 'individual-taxon-canonical-name', 'study-name'], axis='columns')
df = gpd.GeoDataFrame(df, geometry=geometry)
df.crs = "Epsg:4326"
df = df.to_crs(epsg=4326)
df

,timestamp,height-raw,individual-local-identifier,geometry
0,2018-09-13 05:03:05.000,0.0,1208-75545,POINT (-78.87976 51.93200)
1,2018-09-13 05:18:21.000,-4.0,1208-75545,POINT (-78.87949 51.93198)
2,2018-09-13 05:33:34.000,-2.0,1208-75545,POINT (-78.87896 51.93192)
3,2018-09-13 05:48:47.000,-6.0,1208-75545,POINT (-78.87704 51.93223)
4,2018-09-13 06:19:29.000,31.0,1208-75545,POINT (-78.87576 51.93283)
...,...,...,...,...
1681148,2021-02-01 12:08:02.000,40.0,1208-79489,POINT (-75.62054 39.79138)
1681149,2021-02-01 12:22:57.000,811.0,1208-79489,POINT (-75.61787 39.79059)
1681150,2021-02-01 12:38:20.000,88.0,1208-79489,POINT (-75.61929 39.79002)
1681151,2021-02-01 12:52:57.000,63.0,1208-79489,POINT (-75.61935 39.79107)


In [5]:
gdf = df

In [6]:
linestrings = {}
animals = gdf['individual-local-identifier'].unique()
for animal in animals:
    animal_gdf = gdf[gdf['individual-local-identifier'] == animal]
    animal_gdf = animal_gdf.sort_values('timestamp')
    linestrings[animal] = LineString(animal_gdf['geometry'].to_list())


In [7]:
data = {}
data['animal_tag'] = []
data['geometry'] = []
for key, value in linestrings.items():
    data['animal_tag'].append(key)
    data['geometry'].append(value)

In [8]:
gdf = gpd.GeoDataFrame(data)
gdf.crs = "Epsg:4326"
gdf = gdf.to_crs(epsg=4326)
gdf

,animal_tag,geometry
0,1208-75545,"LINESTRING (-78.87976 51.93200, -78.87949 51.9..."
1,1208-75599,"LINESTRING (-78.87221 51.87758, -78.87594 51.8..."
2,1208-75600,"LINESTRING (-78.88998 51.89650, -78.88995 51.8..."
3,1208-75628,"LINESTRING (-78.87453 51.89043, -78.87450 51.8..."
4,1208-75633,"LINESTRING (-78.79884 51.58872, -78.75671 51.4..."
...,...,...
77,1208-79901,"LINESTRING (-69.73338 60.58117, -69.73277 60.5..."
78,1208-79520,"LINESTRING (-69.92357 60.29724, -69.92361 60.2..."
79,1208-79617,"LINESTRING (-69.84627 60.60114, -69.84625 60.6..."
80,1208-79485,"LINESTRING (-70.03949 60.15304, -70.03951 60.1..."


In [16]:
for _, row1 in gdf.iterrows():
    intersections = {}
    intersections['animal_tag1'] = []
    intersections['animal_tag2'] = []
    intersections['geometry'] = []

    for _, row2 in gdf.iterrows():
        if row1.animal_tag != row2.animal_tag:
            intersections['animal_tag1'].append(row1.animal_tag)
            intersections['animal_tag2'].append(row2.animal_tag)
            intersections['geometry'].append(row1.geometry.intersection(row2.geometry))

    intersections_df = pd.DataFrame(intersections)
    intersections_df.to_csv(f"goose_data/{row1.animal_tag}_intersections.csv")

AttributeError: 'Series' object has no attribute 'iterrows'

In [17]:
row1 = gdf.iloc[72]
intersections = {}
intersections['animal_tag1'] = []
intersections['animal_tag2'] = []
intersections['geometry'] = []

for _, row2 in gdf.iterrows():
    if row1.animal_tag != row2.animal_tag:
        intersections['animal_tag1'].append(row1.animal_tag)
        intersections['animal_tag2'].append(row2.animal_tag)
        intersections['geometry'].append(row1.geometry.intersection(row2.geometry))

intersections_df = pd.DataFrame(intersections)
intersections_df.to_csv(f"goose_data/{row1.animal_tag}_intersections.csv")

: 

In [15]:
gdf.iloc[72]

animal_tag                                           1278-00649
geometry      LINESTRING (-69.780182 60.321075, -69.78022 60...
Name: 72, dtype: object